In [1]:
import numpy as np
import glob
import pandas as pd
from monty.serialization import loadfn, dumpfn

from pymatgen.transformations.advanced_transformations import CubicSupercellTransformation
from pymatgen.transformations.standard_transformations import DeformStructureTransformation
from pymatgen.core import Structure


from qubitPack.tool_box import *

/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/pymatgen/ext/matproj.py:454: DeprecationWarning: __init__ is deprecated
MaterialsProjectCompatibility will be updated with new correction classes as well as new values of corrections and uncertainties in 2020
  def get_pourbaix_entries(self, chemsys, solid_compat=MaterialsProjectCompatibility()):
/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/monty/monty/serialization.py:84: PendingDeprecationWarning: 
load will be removed, use

  yaml=YAML(typ='unsafe', pure=True)
  yaml.load(...)

instead
  return yaml.load(fp, *args, **kwargs)
/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/ase/__init__.py:15: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.9':
/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.v

In [4]:

pc_df = pd.DataFrame(columns=['file', 'a', 'b', 'c', 'structure', 'formula'])
for i in glob.glob("pc/original/*.vasp"):
    st = Structure.from_file(i)
    file = i
    formula = st.formula
    #remove space in string formula
    formula = formula.replace(" ", "")
    pc_df = pc_df.append({'file': file, 'a': st.lattice.a, 'b': st.lattice.b, 'c': st.lattice.c, 'structure': st, 'formula': formula}, ignore_index=True)
pc_df.round(2)

,file,a,b,c,structure,formula
0,pc/original/Te2Mo.vasp,3.51,3.51,18.61,"[[ 1.75260975 1.01186765 11.09182808] Te, [1....",Te2Mo1
1,pc/original/MoS2.vasp,3.15,3.15,18.13,"[[0. 0. 9.0635565] Mo, [ 1.57345...",Mo1S2
2,pc/original/WSe2.vasp,3.28,3.28,18.36,"[[0. 0. 9.1792765] W, [ 1.637730...",W1Se2
3,pc/original/MoSe2.vasp,3.28,3.28,18.34,"[[0. 0. 9.1690295] Mo, [ 1.63908...",Mo1Se2
4,pc/original/WS2.vasp,3.15,3.15,18.15,"[[0. 0. 9.0727975] W, [ 1.573725...",W1S2
5,pc/original/Te2W.vasp,3.50,3.50,18.62,"[[ 1.75229309 1.01168499 11.10084577] Te, [1....",Te2W1


# Generate cubic supercell

In [79]:
cubic_sc_df = pd.DataFrame(columns=['formula', 'a', 'b', 'c', 'gamma', 'nsites', 'structure', ])
for i in pc_df['structure']:
    transformer = CubicSupercellTransformation(min_length=20, force_diagonal=True)
    cubic_sc = transformer.apply_transformation(i)
    formula = cubic_sc.formula
    #remove space in string formula
    formula = formula.replace(" ", "")
    cubic_sc_df = cubic_sc_df.append({'formula': formula, 'a': cubic_sc.lattice.a, 'b': cubic_sc.lattice.b, 'c': cubic_sc.lattice.c, 'gamma': cubic_sc.lattice.gamma, 'nsites': cubic_sc.num_sites, 'structure': cubic_sc}, ignore_index=True)
cubic_sc_df.round(2)

# cubic_sc_df.iloc[1]["structure"].to("poscar", "pristine/Mo30S60_eq_ab.vasp")

,formula,a,b,c,gamma,nsites,structure
0,Te144Mo72,21.03,21.03,37.22,120.0,216,"[[ 1.75260975 1.01186765 11.09182808] Te, [ 1..."
1,Mo98S196,22.03,22.03,36.25,120.0,294,"[[0. 0. 9.0635565] Mo, [ 0. ..."
2,W98Se196,22.93,22.93,36.72,120.0,294,"[[0. 0. 9.1792765] W, [ 0. ..."
3,Mo98Se196,22.95,22.95,36.68,120.0,294,"[[0. 0. 9.1690295] Mo, [ 0. ..."
4,W98S196,22.03,22.03,36.29,120.0,294,"[[0. 0. 9.0727975] W, [ 0. ..."
5,Te144W72,21.03,21.03,37.25,120.0,216,"[[ 1.75229309 1.01168499 11.10084577] Te, [ 1..."


In [70]:
cubic_sc_df = pd.DataFrame(columns=['formula', 'a', 'b', 'c', 'gamma', 'nsites', 'structure', ])
for i in pc_df['structure']:
    transformer = CubicSupercellTransformation(min_length=15, force_diagonal=False)
    cubic_sc = transformer.apply_transformation(i)
    formula = cubic_sc.formula
    #remove space in string formula
    formula = formula.replace(" ", "")
    cubic_sc_df = cubic_sc_df.append({'formula': formula, 'a': cubic_sc.lattice.a, 'b': cubic_sc.lattice.b, 'c': cubic_sc.lattice.c, 'gamma': cubic_sc.lattice.gamma, 'nsites': cubic_sc.num_sites, 'structure': cubic_sc}, ignore_index=True)
cubic_sc_df.round(2)

,formula,a,b,c,gamma,nsites,structure
0,Te50Mo25,17.53,15.28,18.61,83.41,75,"[[ 1.75260975 1.01186765 11.09182808] Te, [ 5..."
1,Mo30S60,15.73,16.35,18.13,90.00,90,"[[0. 0. 9.0635565] Mo, [3.146908..."
2,W30Se60,16.38,17.02,18.36,90.00,90,"[[0. 0. 9.1792765] W, [3.275457 ..."
3,Mo30Se60,16.39,17.03,18.34,90.00,90,"[[0. 0. 9.1690295] Mo, [3.278162..."
4,W30S60,15.74,16.35,18.15,90.00,90,"[[0. 0. 9.0727975] W, [3.147447 ..."
5,Te50W25,17.52,15.28,18.62,83.41,75,"[[ 1.75229309 1.01168499 11.10084577] Te, [ 5..."


## Write files

In [65]:
for i in cubic_sc_df['structure']:
    formula = i.formula.replace(" ", "")
    i.to("poscar", filename="cubic_sc/pristine/"+formula+".vasp")


# Generate defect structures

In [66]:
st = Structure.from_file("cubic_sc/strained_pristine/Mo30S60/-0.060.vasp")
st.get_space_group_info()

FileNotFoundError: [Errno 2] No such file or directory: 'cubic_sc/strained_pristine/Mo30S60/-0.060.vasp'

In [ ]:
gen_defect = GenDefect(
    orig_st= st,
    defect_type=("substitutions", 1),
    natom=90,
    vacuum_thickness=20,
    distort=0,
    sub_on_side=None,
    standardize_st=False,
    return_pos2aBR=False,
)
gen_defect.defect_st.to("poscar", "cubic_sc/strained_defect/-0.060.vasp")

In [ ]:
defect_st = Structure.from_file("cubic_sc/strained_defect/-0.060.vasp")
defect_st.get_space_group_info(symprec=0.1)

## A!=B

In [23]:
from monty.serialization import loadfn, dumpfn

mos2_xx = loadfn("pc/strained_pc/xx/-0.008:0.008/Mo1S2.json")
mos2_xx_yy = loadfn("pc/strained_pc/xx_yy/-0.008:0.008/Mo1S2.json")

st_xx = mos2_xx["st"][0].copy()
st_xx_yy = mos2_xx_yy["st"][0].copy()

for st in [st_xx, st_xx_yy]:
    print(st.lattice.abc)
    gen_defect = GenDefect(
    orig_st= st,
    defect_type=("substitutions", 1),
    natom=25*3,
    vacuum_thickness=20,
    distort=0,
    sub_on_side=None,
    standardize_st=False,
    return_pos2aBR=False,
)
    print("-"*10)
    print(gen_defect.defect_st.lattice.abc)


(3.121732736, 3.146907764984541, 18.127113)
Setting up defects...

Number of jobs created:
    bulk = 1
    vacancies:
        vac_1_Mo = 9
        vac_2_S = 5
    substitutions:
        as_1_S_on_Mo = 7
        as_1_Mo_on_S = 7
Total (non dielectric) jobs created = 29

Full Formula (Mo26 S49)
Reduced Formula: Mo26S49
abc   :  15.608664  15.734539  23.090310
angles:  90.000000  90.000000 120.000002
Sites (75)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Mo    0         0         0.5
  1  Mo    0         0.2       0.5
  2  Mo    0         0.4       0.5
  3  Mo    1         0.6       0.5
  4  Mo    0         0.8       0.5
  5  Mo    0.2       0         0.5
  6  Mo    0.2       0.2       0.5
  7  Mo    0.2       0.4       0.5
  8  Mo    0.2       0.6       0.5
  9  Mo    0.2       0.8       0.5
 10  Mo    0.4       0         0.5
 11  Mo    0.4       0.2       0.5
 12  Mo    0.4       0.4       0.5
 13  Mo    0.4       0.6       0.5
 14  Mo    0.4   

# Generate strained structures

In [2]:
# get a list between -0.01 and 0.01 with 9 segments

def gen_strained_sts(orig_df, write_file, strained_setting=None):
    orig_df = orig_df.copy()
    strained_df = pd.DataFrame(columns=['formula', 'strain', 'a', 'b', 'c', 'gamma', 'nsites', 'spg', 'structure', ])
    # strain_range = np.linspace(-0.01, 0.01, 11)
    # from -0.06 to 0.06 with 13 segments
    strained_setting = strained_setting or {}
    strain_min = strained_setting.get("strain_min", -0.06)
    strain_max = strained_setting.get("strain_max", 0.06)
    strain_step = strained_setting.get("strain_step", 0.01)
    strain_type = strained_setting.get("strain_type", "xx_yy")
    strain_range = np.linspace(strain_min, strain_max, int((strain_max-strain_min)/strain_step)+1)
    if strained_setting.get("exclude_zero", False):
        strain_range = np.delete(strain_range, np.where(strain_range==0))

    print("strain_range: ", strain_range)
    for formula in orig_df['formula']:
        print("=====================================")
        print(f"formula: {formula}")
        # create a folder for each formula
        # os.makedirs("cubic_sc/strained_pristine/"+formula, exist_ok=True)
        st_info_json = {'formula': formula, 'strain': [], 'st': []}
        for strain in strain_range:
            st = orig_df[orig_df["formula"]==formula]["structure"].values[0].copy()
            print(f"before strain: {st.lattice.a:.2f}, {st.lattice.b:.2f}, {st.lattice.c:.2f}")

            # strain_matrix = [strain, strain, 0] if strain_type == "xx_yy" else [0, 0, strain]
            strain_matrix = []
            if strain_type == "xx_yy":
                strain_matrix = [strain, strain, 0]
            elif strain_type == "xx":
                strain_matrix = [strain, 0, 0]
            elif strain_type == "yy":
                strain_matrix = [0, strain, 0]
            else:
                strain_matrix = [0, 0, strain]
            st.apply_strain(strain_matrix)
            print(f"after strain: {st.lattice.a:.2f}, {st.lattice.b:.2f}, {st.lattice.c:.2f}, strain: {strain: .3f}")
            print(f"spg: {st.get_space_group_info()}")
            strained_df = strained_df.append({'formula': formula, 'strain': strain, 'a': st.lattice.a, 'b': st.lattice.b, 'c': st.lattice.c, 'gamma': st.lattice.gamma, 'nsites': st.num_sites, 'spg': st.get_space_group_info(), 'structure': st}, ignore_index=True)

            st_info_json['strain'].append(round(strain, 3))
            st_info_json['st'].append(st.as_dict())

        if write_file:
            os.makedirs(write_file+f"/{strain_type}/{strain_min}:{strain_max}/", exist_ok=True)
            dumpfn(st_info_json, write_file+f"/{strain_type}/{strain_min}:{strain_max}/"+formula+".json", indent=4)

    return strained_df


## cubic supercell

In [ ]:
# get a list between -0.01 and 0.01 with 9 segments
strained_df = pd.DataFrame(columns=['formula', 'strain', 'a', 'b', 'c', 'gamma', 'nsites', 'spg', 'structure', ])
# strain_range = np.linspace(-0.01, 0.01, 11)
# from -0.06 to 0.06 with 13 segments
strain_range = np.linspace(-0.06, 0.06, 13)
for formula in cubic_sc_df['formula']:
    if "Te" in formula:
        continue
    print("=====================================")
    print(f"formula: {formula}")
    # create a folder for each formula
    # os.makedirs("cubic_sc/strained_pristine/"+formula, exist_ok=True)
    st_info_json = {'formula': formula, 'strain': [], 'st': []}
    for strain in strain_range:
        st = cubic_sc_df[cubic_sc_df["formula"]==formula]["structure"].values[0].copy()
        print(f"before strain: {st.lattice.a:.2f}, {st.lattice.b:.2f}, {st.lattice.c:.2f}")

        st.apply_strain([strain, strain, 0])
        # st.apply_strain([0, 0, strain])
        print(f"after strain: {st.lattice.a:.2f}, {st.lattice.b:.2f}, {st.lattice.c:.2f}, strain: {strain: .3f}")
        print(f"spg: {st.get_space_group_info()}")
        strained_df = strained_df.append({'formula': formula, 'strain': strain, 'a': st.lattice.a, 'b': st.lattice.b, 'c': st.lattice.c, 'gamma': st.lattice.gamma, 'nsites': st.num_sites, 'spg': st.get_space_group_info(), 'structure': st}, ignore_index=True)

        st_info_json['strain'].append(round(strain, 3))
        st_info_json['st'].append(st.as_dict())
    # dumpfn(st_info_json, "cubic_sc/strained_pristine/zz/"+formula+".json", indent=4)



## Primitive cell

In [7]:
# gen_strained_sts(pc_df, "pc/strained_pc/")
gen_strained_sts(pc_df, "pc/strained_pc/", strained_setting={"strain_min": -0.008, "strain_max": 0.008, "strain_step": 0.002,
                                                "exclude_zero": False, 'strain_type': "xx"})


strain_range:  [-0.008 -0.006 -0.004 -0.002  0.     0.002  0.004  0.006  0.008]
formula: Te2Mo1
before strain: 3.51, 3.51, 18.61
after strain: 3.48, 3.51, 18.61, strain: -0.008
spg: ('Pm', 6)
before strain: 3.51, 3.51, 18.61
after strain: 3.48, 3.51, 18.61, strain: -0.006
spg: ('Pm', 6)
before strain: 3.51, 3.51, 18.61
after strain: 3.49, 3.51, 18.61, strain: -0.004
spg: ('Pm', 6)
before strain: 3.51, 3.51, 18.61
after strain: 3.50, 3.51, 18.61, strain: -0.002
spg: ('P-6m2', 187)
before strain: 3.51, 3.51, 18.61
after strain: 3.51, 3.51, 18.61, strain:  0.000
spg: ('P-6m2', 187)
before strain: 3.51, 3.51, 18.61
after strain: 3.51, 3.51, 18.61, strain:  0.002
spg: ('P-6m2', 187)
before strain: 3.51, 3.51, 18.61
after strain: 3.52, 3.51, 18.61, strain:  0.004
spg: ('Pm', 6)
before strain: 3.51, 3.51, 18.61
after strain: 3.53, 3.51, 18.61, strain:  0.006
spg: ('Pm', 6)
before strain: 3.51, 3.51, 18.61
after strain: 3.53, 3.51, 18.61, strain:  0.008
spg: ('Pm', 6)
formula: Mo1S2
before str

,formula,strain,a,b,c,gamma,nsites,spg,structure
0,Te2Mo1,-0.008,3.477174,3.505216,18.610200,120.000001,3,"(Pm, 6)","[[ 1.73391526 1.01186765 11.09182808] Te, [1...."
1,Te2Mo1,-0.006,3.484185,3.505216,18.610200,120.000001,3,"(Pm, 6)","[[ 1.73858888 1.01186765 11.09182808] Te, [1...."
2,Te2Mo1,-0.004,3.491195,3.505216,18.610200,120.000001,3,"(Pm, 6)","[[ 1.74326251 1.01186765 11.09182808] Te, [1...."
3,Te2Mo1,-0.002,3.498206,3.505216,18.610200,120.000001,3,"(P-6m2, 187)","[[ 1.74793613 1.01186765 11.09182808] Te, [1...."
4,Te2Mo1,0.000,3.505216,3.505216,18.610200,120.000001,3,"(P-6m2, 187)","[[ 1.75260975 1.01186765 11.09182808] Te, [1...."
5,Te2Mo1,0.002,3.512226,3.505216,18.610200,120.000001,3,"(P-6m2, 187)","[[ 1.75728338 1.01186765 11.09182808] Te, [1...."
6,Te2Mo1,0.004,3.519237,3.505216,18.610200,120.000001,3,"(Pm, 6)","[[ 1.761957 1.01186765 11.09182808] Te, [1...."
7,Te2Mo1,0.006,3.526247,3.505216,18.610200,120.000001,3,"(Pm, 6)","[[ 1.76663062 1.01186765 11.09182808] Te, [1...."
8,Te2Mo1,0.008,3.533258,3.505216,18.610200,120.000001,3,"(Pm, 6)","[[ 1.77130425 1.01186765 11.09182808] Te, [1...."
9,Mo1S2,-0.008,3.121733,3.146908,18.127113,120.000002,3,"(Pm, 6)","[[0. 0. 9.0635565] Mo, [ 1.55667..."


## Test if json file is correct

In [25]:
# read st_info_json
for i in glob.glob("pc/strained_pc/xx_yy/-0.008:0.008/*.json"):
    st_info_json = loadfn(i)
    print(i)
    df = pd.DataFrame(st_info_json)
    df["a"] = df["st"].apply(lambda x: x.lattice.a)
    df["b"] = df["st"].apply(lambda x: x.lattice.b)
    df["c"] = df["st"].apply(lambda x: x.lattice.c)
    display(df.loc[:].round(3))

pc/strained_pc/xx_yy/-0.008:0.008/Mo1S2.json


,formula,strain,st,a,b,c
0,Mo1S2,-0.008,"[[0. 0. 9.0635565] Mo, [ 1.56086...",3.122,3.122,18.127
1,Mo1S2,-0.006,"[[0. 0. 9.0635565] Mo, [ 1.56401...",3.128,3.128,18.127
2,Mo1S2,-0.004,"[[0. 0. 9.0635565] Mo, [ 1.56716...",3.134,3.134,18.127
3,Mo1S2,-0.002,"[[0. 0. 9.0635565] Mo, [ 1.57030...",3.141,3.141,18.127
4,Mo1S2,0.002,"[[0. 0. 9.0635565] Mo, [ 1.57660...",3.153,3.153,18.127
5,Mo1S2,0.004,"[[0. 0. 9.0635565] Mo, [ 1.57974...",3.159,3.159,18.127
6,Mo1S2,0.006,"[[0. 0. 9.0635565] Mo, [ 1.58289...",3.166,3.166,18.127
7,Mo1S2,0.008,"[[0. 0. 9.0635565] Mo, [ 1.58604...",3.172,3.172,18.127


pc/strained_pc/xx_yy/-0.008:0.008/Mo1Se2.json


,formula,strain,st,a,b,c
0,Mo1Se2,-0.008,"[[0. 0. 9.1690295] Mo, [ 1.62597...",3.252,3.252,18.338
1,Mo1Se2,-0.006,"[[0. 0. 9.1690295] Mo, [ 1.62924...",3.258,3.258,18.338
2,Mo1Se2,-0.004,"[[0. 0. 9.1690295] Mo, [ 1.63252...",3.265,3.265,18.338
3,Mo1Se2,-0.002,"[[0. 0. 9.1690295] Mo, [ 1.63580...",3.272,3.272,18.338
4,Mo1Se2,0.002,"[[0. 0. 9.1690295] Mo, [ 1.64236...",3.285,3.285,18.338
5,Mo1Se2,0.004,"[[0. 0. 9.1690295] Mo, [ 1.64563...",3.291,3.291,18.338
6,Mo1Se2,0.006,"[[0. 0. 9.1690295] Mo, [ 1.64891...",3.298,3.298,18.338
7,Mo1Se2,0.008,"[[0. 0. 9.1690295] Mo, [ 1.65219...",3.304,3.304,18.338


pc/strained_pc/xx_yy/-0.008:0.008/W1Se2.json


,formula,strain,st,a,b,c
0,W1Se2,-0.008,"[[0. 0. 9.1792765] W, [ 1.624628...",3.249,3.249,18.359
1,W1Se2,-0.006,"[[0. 0. 9.1792765] W, [ 1.627903...",3.256,3.256,18.359
2,W1Se2,-0.004,"[[0. 0. 9.1792765] W, [ 1.631179...",3.262,3.262,18.359
3,W1Se2,-0.002,"[[0. 0. 9.1792765] W, [ 1.634454...",3.269,3.269,18.359
4,W1Se2,0.002,"[[0. 0. 9.1792765] W, [ 1.641005...",3.282,3.282,18.359
5,W1Se2,0.004,"[[0. 0. 9.1792765] W, [ 1.644281...",3.289,3.289,18.359
6,W1Se2,0.006,"[[0. 0. 9.1792765] W, [ 1.647556...",3.295,3.295,18.359
7,W1Se2,0.008,"[[0. 0. 9.1792765] W, [ 1.650832...",3.302,3.302,18.359


pc/strained_pc/xx_yy/-0.008:0.008/W1S2.json


,formula,strain,st,a,b,c
0,W1S2,-0.008,"[[0. 0. 9.0727975] W, [ 1.561135...",3.122,3.122,18.146
1,W1S2,-0.006,"[[0. 0. 9.0727975] W, [ 1.564282...",3.129,3.129,18.146
2,W1S2,-0.004,"[[0. 0. 9.0727975] W, [ 1.567430...",3.135,3.135,18.146
3,W1S2,-0.002,"[[0. 0. 9.0727975] W, [ 1.570577...",3.141,3.141,18.146
4,W1S2,0.002,"[[0. 0. 9.0727975] W, [ 1.576872...",3.154,3.154,18.146
5,W1S2,0.004,"[[0. 0. 9.0727975] W, [ 1.580020...",3.160,3.160,18.146
6,W1S2,0.006,"[[0. 0. 9.0727975] W, [ 1.583167...",3.166,3.166,18.146
7,W1S2,0.008,"[[0. 0. 9.0727975] W, [ 1.586315...",3.173,3.173,18.146


In [23]:
for i in strained_df["formula"].unique():
    display(strained_df[strained_df["formula"]==i].round(3))

NameError: name 'strained_df' is not defined

# Test if the shear strain happens in cubic supercell and in triangular supercell

In [137]:
from pymatgen.core import Lattice
def another_way_to_add_strain(pc, biaxial_ratio):
    pc = pc.copy()
    new_lattice = Lattice(pc.lattice.matrix.dot(np.eye(3)*(1+biaxial_ratio)))
    # pc = Structure(new_lattice, pc.species, pc.cart_coords, coords_are_cartesian=True)
    pc.lattice = new_lattice
    return pc

In [138]:
tri_lattice = Structure.from_file("cubic_sc/pristine/Mo30S60_eq_ab.vasp")
cubic_lattice = Structure.from_file("cubic_sc/pristine/Mo30S60.vasp")

print(f"gamma: {tri_lattice.lattice.gamma:.2f}, {cubic_lattice.lattice.gamma:.2f}")



angle_cubic = cubic_lattice.get_angle(11, 41, 17)
angle_tri = tri_lattice.get_angle(12, 37, 17)
distance_cubic = cubic_lattice.get_distance(11, 41)
distance_tri = tri_lattice.get_distance(12, 37)


print(f"before strain of cubic: {angle_cubic:.2f}, {distance_cubic:.2f}")
print(f"before strain of tri: {angle_tri:.2f}, {distance_tri:.2f}")

strain = 0.06
cubic_lattice.apply_strain([strain, strain, 0])
tri_lattice.apply_strain([strain, strain, 0])
cubic_lattice_way = another_way_to_add_strain(cubic_lattice, strain)
tri_lattice_way = another_way_to_add_strain(tri_lattice, strain)

#print if cubic_lattice == cubic_lattice_way and tri_lattice == tri_lattice_way
print(f"cubic_lattice == cubic_lattice_way: {Structure.__eq__(cubic_lattice, cubic_lattice_way)}")
print(f"tri_lattice == tri_lattice_way: {Structure.__eq__(tri_lattice, tri_lattice_way)}")



angle_cubic = cubic_lattice.get_angle(11, 41, 17)
angle_tri = tri_lattice.get_angle(12, 37, 17)
distance_cubic = cubic_lattice.get_distance(11, 41)
distance_tri = tri_lattice.get_distance(12, 37)

angle_cubic_way = cubic_lattice_way.get_angle(11, 41, 17)
angle_tri_way = tri_lattice_way.get_angle(12, 37, 17)
distance_cubic_way = cubic_lattice_way.get_distance(11, 41)
distance_tri_way = tri_lattice_way.get_distance(12, 37)

print(f"after strain of cubic: angles: {angle_cubic:.2f}, {angle_cubic_way:.2f}, distances: {distance_cubic:.2f}, {distance_cubic_way:.2f}")
print(f"after strain of tri: angles: {angle_tri:.2f}, {angle_tri_way:.2f}, distances: {distance_tri:.2f}, {distance_tri_way:.2f}")


gamma: 120.00, 90.00
before strain of cubic: 82.56, 2.39
before strain of tri: 82.56, 2.39
cubic_lattice == cubic_lattice_way: False
tri_lattice == tri_lattice_way: False
after strain of cubic: angles: 84.98, 84.98, distances: 2.47, 2.62
after strain of tri: angles: 84.98, 84.98, distances: 2.47, 2.62


# Generate bilayer

## Read 3D primitive cell

In [5]:
pc_df = pd.DataFrame(columns=['file', 'a', 'b', 'c', 'formula', 'structure', 'nsites'])
for i in glob.glob("pc/original/3D_TMDs/*.vasp"):
    st = Structure.from_file(i)
    file = i
    formula = st.formula
    nsites = st.num_sites
    #remove space in string formula
    formula = formula.replace(" ", "")
    pc_df = pc_df.append({'file': file, 'a': st.lattice.a, 'b': st.lattice.b, 'c': st.lattice.c, 'structure': st,
                          'formula': formula, 'nsites': nsites}, ignore_index=True)
pc_df.round(2)

,file,a,b,c,formula,structure,nsites
0,pc/original/3D_TMDs/MoS2.vasp,3.19,3.19,13.38,Mo2S4,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1...",6
1,pc/original/3D_TMDs/WSe2.vasp,3.32,3.32,13.74,W2Se4,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6...",6
2,pc/original/3D_TMDs/MoSe2.vasp,3.32,3.32,13.54,Mo2Se4,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1....",6
3,pc/original/3D_TMDs/WS2.vasp,3.18,3.18,12.98,W2S4,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592...",6


## create defect for testing

In [122]:
for i in pc_df["formula"].unique():
    gen_defect = GenDefect(
        orig_st= pc_df.loc[pc_df["formula"]==i]["structure"].iloc[0],
        defect_type=("substitutions", 1),
        natom=6*25,
        vacuum_thickness=20,
        distort=0,
        sub_on_side=None,
        standardize_st=False,
        return_pos2aBR=False,
    )
    gen_defect.defect_st.to("poscar", "pc/original/3D_TMDs/defect_test/"+i+".vasp")

/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/MPInterfaces/mpinterfaces/utils.py:817: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if axis == 'c' and direction == (0, 0, 1):
/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/spglib/spglib.py:151: DeprecationWarning: Use get_magnetic_symmetry() for cell with magnetic moments.
  DeprecationWarning,


Setting up defects...

Number of jobs created:
    bulk = 1
    vacancies:
        vac_1_Mo = 9
        vac_2_S = 5
        vac_3_S = 5
    substitutions:
        as_1_S_on_Mo = 7
        as_1_Mo_on_S = 7
        as_2_Mo_on_S = 7
Total (non dielectric) jobs created = 41

Full Formula (Mo51 S99)
Reduced Formula: Mo17S33
abc   :  15.961190  15.961190  29.817474
angles:  90.000000  90.000000 120.000000
Sites (150)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Mo    0.133333  0.066667  0.387832
  1  Mo    0.133333  0.266667  0.387832
  2  Mo    0.133333  0.466667  0.387832
  3  Mo    0.133333  0.666667  0.387832
  4  Mo    0.133333  0.866667  0.387832
  5  Mo    0.333333  0.066667  0.387832
  6  Mo    0.333333  0.266667  0.387832
  7  Mo    0.333333  0.466667  0.387832
  8  Mo    0.333333  0.666667  0.387832
  9  Mo    0.333333  0.866667  0.387832
 10  Mo    0.533333  0.066667  0.387832
 11  Mo    0.533333  0.266667  0.387832
 12  Mo    0.533333  0.46

/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/pymatgen/analysis/local_env.py:4303: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  "No oxidation states specified on sites! For better results, set "
/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/pymatgen/analysis/local_env.py:4086: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  "CrystalNN: cannot locate an appropriate radius, "
/home/tsai/.conda/envs/workflow/lib/python3.7/site-packages/pymatgen/core/lattice.py:1364: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release no

Setting up defects...

Number of jobs created:
    bulk = 1
    vacancies:
        vac_1_W = 9
        vac_2_Se = 5
        vac_3_Se = 5
    substitutions:
        as_1_W_on_Se = 7
        as_2_W_on_Se = 7
        as_1_Se_on_W = 7
Total (non dielectric) jobs created = 41

Full Formula (W51 Se99)
Reduced Formula: W17Se33
abc   :  16.599665  16.599665  30.193015
angles:  90.000000  90.000000 120.000000
Sites (150)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  W     0.133333  0.066667  0.613745
  1  W     0.133333  0.266667  0.613745
  2  W     0.133333  0.466667  0.613745
  3  W     0.133333  0.666667  0.613745
  4  W     0.133333  0.866667  0.613745
  5  W     0.333333  0.066667  0.613745
  6  W     0.333333  0.266667  0.613745
  7  W     0.333333  0.466667  0.613745
  8  W     0.333333  0.666667  0.613745
  9  W     0.333333  0.866667  0.613745
 10  W     0.533333  0.066667  0.613745
 11  W     0.533333  0.266667  0.613745
 12  W     0.533333  0.4

## create z-strain by translating one layer along z-axis

In [94]:
st = pc_df.iloc[0]["structure"]

# find sites whose species are S and sort them by z coordinate
s_sites = sorted([i for i in st.sites if i.specie.name == "S"], key=lambda x: x.coords[2])
print(f"S sites: {s_sites}")
# find z-coordinate difference between the 1st and 2nd S sites
z_diff = s_sites[2].coords[2] - s_sites[1].coords[2]
print(f"z_diff: {z_diff:.3f}, 1th z: {s_sites[2].coords[2]:.3f}, 2nd z: {s_sites[1].coords[2]:.3f}")

for i in st.sites:
    print(i.specie, i.coords)
# translate the bottom layer upward along z-axis
from qubitPack.tool_box import move_site
# find those sites whose z coordinate is smaller than 6 in cartesian coordinate
sites_to_move = [idx for idx, i in enumerate(st.sites) if i.coords[2] < 8]
print(f"number of sites to move: {len(sites_to_move)}")
# move the sites upward by 0.1 angstrom
modified_st = st.copy()
modified_st.translate_sites(sites_to_move, [0, 0, 0.5], frac_coords=False)

for idx, i in enumerate(modified_st.sites):
    print(f"{st.sites[idx].specie}: original z:{st.sites[idx].coords[2]:.2f}, modified z: {i.coords[2]:.2f}({i.coords[2] - st.sites[idx].coords[2]:.2f})")

# modified_st.to("poscar", "pc/original/3D_TMDs/0.5.vasp")

S sites: [PeriodicSite: S (1.5961, 0.9215, 1.7804) [0.3333, 0.6667, 0.1331], PeriodicSite: S (1.5961, 0.9215, 4.9087) [0.3333, 0.6667, 0.3669], PeriodicSite: S (1.5961, -0.9215, 8.4696) [0.6667, 0.3333, 0.6331], PeriodicSite: S (1.5961, -0.9215, 11.5979) [0.6667, 0.3333, 0.8669]]
z_diff: 3.561, 1th z: 8.470, 2nd z: 4.909
Mo [ 1.59611896 -0.92151971  3.3445735 ]
Mo [ 1.59611896  0.92151971 10.0337205 ]
S [ 1.59611896 -0.92151971  8.46955712]
S [1.59611896 0.92151971 4.90873688]
S [ 1.59611896 -0.92151971 11.59788388]
S [1.59611896 0.92151971 1.78041012]
number of sites to move: 3
Mo: original z:3.34, modified z: 3.84(0.50)
Mo: original z:10.03, modified z: 10.03(0.00)
S: original z:8.47, modified z: 8.47(0.00)
S: original z:4.91, modified z: 5.41(0.50)
S: original z:11.60, modified z: 11.60(0.00)
S: original z:1.78, modified z: 2.28(0.50)


In [6]:
# get a list between -0.01 and 0.01 with 9 segments

def gen_strained_sts(orig_df, write_file, strained_setting=None):
    orig_df = orig_df.copy()
    strained_df = pd.DataFrame(columns=['formula', 'strain', 'a', 'b', 'c', 'gamma', 'nsites', 'spg', 'z_diff', 'strain_type', 'structure'])
    # strain_range = np.linspace(-0.01, 0.01, 11)
    # from -0.06 to 0.06 with 13 segments
    strained_setting = strained_setting or {}
    strain_min = strained_setting.get("strain_min", -0.06)
    strain_max = strained_setting.get("strain_max", 0.06)
    strain_step = strained_setting.get("strain_step", 0.01)
    strain_type = strained_setting.get("strain_type", "zz")
    strain_range = np.linspace(strain_min, strain_max, int((strain_max-strain_min)/strain_step)+1)
    if strained_setting.get("exclude_zero", False):
        strain_range = np.delete(strain_range, np.where(strain_range==0))

    print("strain_range: ", strain_range)
    for formula in orig_df['formula']:
        print("=====================================")
        print(f"formula: {formula}")
        # create a folder for each formula
        # os.makedirs("cubic_sc/strained_pristine/"+formula, exist_ok=True)
        st_info_json = {'formula': formula, 'strain': [], 'st': []}
        for strain in strain_range:
            st = orig_df[orig_df["formula"]==formula]["structure"].values[0].copy()
            print(f"before strain: {st.lattice.a:.2f}, {st.lattice.b:.2f}, {st.lattice.c:.2f}")

            anion_sites = sorted([i for i in st.sites if (i.specie.name == "S") or (i.specie.name=="Se")], key=lambda x: x.coords[2])
            print(f"anion_sites: {anion_sites}")
            # find z-coordinate difference between the 1st and 2nd S sites
            z_diff = anion_sites[2].coords[2] - anion_sites[1].coords[2]
            print(f"z_diff: {z_diff:.3f}, 1th z: {anion_sites[2].coords[2]:.3f}, 2nd z: {anion_sites[1].coords[2]:.3f}")

            for i in st.sites:
                print(i.specie, i.coords)
            # translate the bottom layer upward along z-axis
            from qubitPack.tool_box import move_site
            # find those sites whose z coordinate is smaller than 6 in cartesian coordinate
            sites_to_move = [idx for idx, i in enumerate(st.sites) if i.coords[2] < 8]
            print(f"number of sites to move: {len(sites_to_move)}")
            # move the sites upward by 0.1 angstrom
            modified_st = st.copy()
            modified_st.translate_sites(sites_to_move, [0, 0, strain*z_diff], frac_coords=False)
            anion_sites = sorted([i for i in modified_st.sites if (i.specie.name == "S") or (i.specie.name=="Se")], key=lambda x: x.coords[2])
            z_diff = anion_sites[2].coords[2] - anion_sites[1].coords[2]
            print(f"z_diff: {z_diff:.3f}, 1th z: {anion_sites[2].coords[2]:.3f}, 2nd z: {anion_sites[1].coords[2]:.3f}")

            for idx, j in enumerate(modified_st.sites):
                print(f"{st.sites[idx].specie}: original z:{st.sites[idx].coords[2]:.2f}, modified z: {j.coords[2]:.2f}({j.coords[2] - st.sites[idx].coords[2]:.2f})")

            strained_df = strained_df.append({'formula': formula, 'strain': strain, 'a': modified_st.lattice.a, 'b': modified_st
                                             .lattice.b, 'c': modified_st.lattice.c, 'gamma': modified_st.lattice.gamma, 'nsites': modified_st
                                             .num_sites, 'spg': modified_st.get_space_group_info(), 'structure': modified_st, "z_diff":
                                             z_diff, "strain_type": strain_type},
                                             ignore_index=True)

            st_info_json['strain'].append(round(strain, 3))
            st_info_json['st'].append(modified_st.as_dict())

        if write_file:
            os.makedirs(write_file+f"/{strain_type}/{strain_min}:{strain_max}/", exist_ok=True)
            dumpfn(st_info_json, write_file+f"/{strain_type}/{strain_min}:{strain_max}/"+formula+".json", indent=4)

    return strained_df

In [48]:
strained_df_copy = gen_strained_sts(pc_df, 'pc/strained_pc/3D_TMDs', {"strain_min": 0.3, "strain_max": 0.6,
                                                                    "strain_step": 0.1,
                                            "exclude_zero": False})

# strained_df_copy = gen_strained_sts(pc_df, None, {"strain_min": 0.3, "strain_max": 0.6,
#                                                                     "strain_step": 0.1,
#                                             "exclude_zero": False})

strain_range:  [0.3  0.45 0.6 ]
formula: Mo2S4
before strain: 3.19, 3.19, 13.38
anion_sites: [PeriodicSite: S (1.5961, 0.9215, 1.7804) [0.3333, 0.6667, 0.1331], PeriodicSite: S (1.5961, 0.9215, 4.9087) [0.3333, 0.6667, 0.3669], PeriodicSite: S (1.5961, -0.9215, 8.4696) [0.6667, 0.3333, 0.6331], PeriodicSite: S (1.5961, -0.9215, 11.5979) [0.6667, 0.3333, 0.8669]]
z_diff: 3.561, 1th z: 8.470, 2nd z: 4.909
Mo [ 1.59611896 -0.92151971  3.3445735 ]
Mo [ 1.59611896  0.92151971 10.0337205 ]
S [ 1.59611896 -0.92151971  8.46955712]
S [1.59611896 0.92151971 4.90873688]
S [ 1.59611896 -0.92151971 11.59788388]
S [1.59611896 0.92151971 1.78041012]
number of sites to move: 3
z_diff: 2.493, 1th z: 8.470, 2nd z: 5.977
Mo: original z:3.34, modified z: 4.41(1.07)
Mo: original z:10.03, modified z: 10.03(0.00)
S: original z:8.47, modified z: 8.47(0.00)
S: original z:4.91, modified z: 5.98(1.07)
S: original z:11.60, modified z: 11.60(0.00)
S: original z:1.78, modified z: 2.85(1.07)
before strain: 3.19, 3.1

In [47]:
strained_df_copy.loc[strained_df_copy.formula.str.contains("Mo2S4")]
# strained_df_copy.iloc[3]["structure"].to("poscar", "pc/strained_pc/3D_TMDs/zz/Mo2S4_2.vasp")

,formula,strain,a,b,c,gamma,nsites,spg,z_diff,strain_type,structure
0,Mo2S4,0.30,3.192238,3.192238,13.378294,120.0,6,"(P-3m1, 164)",2.492574,zz,"[[ 1.59611896 -0.92151971 4.41281957] Mo, [ 1..."
1,Mo2S4,0.45,3.192238,3.192238,13.378294,120.0,6,"(P-3m1, 164)",1.958451,zz,"[[ 1.59611896 -0.92151971 4.94694261] Mo, [ 1..."
2,Mo2S4,0.60,3.192238,3.192238,13.378294,120.0,6,"(P-3m1, 164)",1.424328,zz,"[[ 1.59611896 -0.92151971 5.48106565] Mo, [ 1..."


### test json

In [116]:
# read st_info_json
for i in glob.glob("pc/strained_pc/3D_TMDs/zz/-0.06:0.06/*.json"):
    st_info_json = loadfn(i)
    print(i)
    df = pd.DataFrame(st_info_json)
    display(df.loc[:].round(3))

pc/strained_pc/3D_TMDs/zz/-0.06:0.06/Mo2Se4.json


,formula,strain,st
0,Mo2Se4,-0.06,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
1,Mo2Se4,-0.05,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
2,Mo2Se4,-0.04,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
3,Mo2Se4,-0.03,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
4,Mo2Se4,-0.02,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
5,Mo2Se4,-0.01,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
6,Mo2Se4,0.00,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
7,Mo2Se4,0.01,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
8,Mo2Se4,0.02,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."
9,Mo2Se4,0.03,"[[ 1.66113009 -0.95905391 10.15721741] Mo, [1...."


pc/strained_pc/3D_TMDs/zz/-0.06:0.06/W2Se4.json


,formula,strain,st
0,W2Se4,-0.06,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
1,W2Se4,-0.05,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
2,W2Se4,-0.04,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
3,W2Se4,-0.03,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
4,W2Se4,-0.02,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
5,W2Se4,-0.01,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
6,W2Se4,0.00,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
7,W2Se4,0.01,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
8,W2Se4,0.02,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."
9,W2Se4,0.03,"[[ 1.65996646 -0.95838208 10.30289047] W, [1.6..."


pc/strained_pc/3D_TMDs/zz/-0.06:0.06/W2S4.json


,formula,strain,st
0,W2S4,-0.06,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
1,W2S4,-0.05,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
2,W2S4,-0.04,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
3,W2S4,-0.03,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
4,W2S4,-0.02,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
5,W2S4,-0.01,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
6,W2S4,0.00,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
7,W2S4,0.01,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
8,W2S4,0.02,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."
9,W2S4,0.03,"[[1.59211145 0.91920597 3.24457059] W, [ 1.592..."


pc/strained_pc/3D_TMDs/zz/-0.06:0.06/Mo2S4.json


,formula,strain,st
0,Mo2S4,-0.06,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
1,Mo2S4,-0.05,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
2,Mo2S4,-0.04,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
3,Mo2S4,-0.03,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
4,Mo2S4,-0.02,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
5,Mo2S4,-0.01,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
6,Mo2S4,0.00,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
7,Mo2S4,0.01,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
8,Mo2S4,0.02,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
9,Mo2S4,0.03,"[[ 1.59611896 -0.92151971 3.3445735 ] Mo, [ 1..."
